In [13]:
import pandas as pd
import numpy as np
import random
import os
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

from numpy import hstack, vstack
import itertools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from itertools import product

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings(action='ignore')

In [14]:
def find_directory(foldername, filename = None, back_num = 0):
    cur = os.getcwd()
    for i in range(back_num):
        cur = os.path.abspath(os.path.join(cur, os.pardir))
    for folder in foldername:
        cur = os.path.join(cur, folder)
    if not os.path.exists(cur):
        os.makedirs(cur)
        print(f'{cur} created')
    if filename != None:
        cur = os.path.join(cur, filename)
    return cur

os.getcwd()

'c:\\Users\\dkstj\\Desktop\\연구\\현대차 3차\\Rapid_fin\\deep learning modeling'

In [15]:
csv_add = find_directory(foldername = [], filename = 'SOC_Point_Data.csv')
dat = pd.read_csv(csv_add, index_col = (0,1,2,3,4))

In [16]:
def Get_Data(dat) :
    
    RPT_MODE = "0.1C"
    SOC_Range = [9,10,11,12]

    Time_Range = range(6, 15, 2)
    SOC_Range = [str(i) for i in SOC_Range]
    

    Data = dat

    X = Data.loc[RPT_MODE, "0" : "16"]
    Y = Data.loc[RPT_MODE, ["SOH", "Next_SOH", "Ratio_SOH", "Ratio_CYC"]].groupby(level = ["Next", "Path", "Number"]).mean()
    
    y = pd.Series(Y["Next_SOH"] - Y["SOH"], name = "Delta_SOH")
    
    Y = pd.concat([Y, y], axis = 1)

    X_seek = X.loc[X.index.get_level_values("Time").isin(Time_Range), SOC_Range]


    X_std = X_seek.groupby(level = ["Next", "Path", "Number"]).std()
    
    return X_std, Y

In [17]:
def Even_Split(X, y, no_next, test_size, rs) :

    Nexts = [n for n in ['M', 'D', 'H'] if n not in no_next]

    XX = {n: X.xs(key = n, level = 'Next', drop_level = False) for n in Nexts}
    yy = {n: y.xs(key = n, level = 'Next', drop_level = False) for n in Nexts}
    
    
    XXX = {n: [] for n in Nexts}
    yyy = {n: [] for n in Nexts}
    
    
    for n in Nexts:
        for path in range(1,5) :
            X_path = XX[n].loc[XX[n].index.get_level_values(level = 'Path').str.len() == path]
            y_path = yy[n].loc[yy[n].index.get_level_values(level = 'Path').str.len() == path]
            
            XXX[n].append(X_path)
            yyy[n].append(y_path)
            
            
    XX_tn = {n: [] for n in Nexts}
    XX_te = {n: [] for n in Nexts}
    
    yy_tn = {n: [] for n in Nexts}
    yy_te = {n: [] for n in Nexts}
        
    for n in Nexts :
        for path in range(1,5) :
            X_temp = XXX[n][path-1]
            y_temp = yyy[n][path-1]
            
            X_tn, X_te, y_tn, y_te = train_test_split(X_temp, y_temp, test_size = test_size, random_state = rs)
            
            XX_tn[n].append(X_tn)
            XX_te[n].append(X_te)
            yy_tn[n].append(y_tn)
            yy_te[n].append(y_te)
                  
    for n in Nexts :
        XX_tn[n] = pd.concat(XX_tn[n])
        XX_te[n] = pd.concat(XX_te[n])
        yy_tn[n] = pd.concat(yy_tn[n])
        yy_te[n] = pd.concat(yy_te[n])
        
        
    X_tn = pd.concat(XX_tn.values())
    X_te = pd.concat(XX_te.values())
    
    y_tn = pd.concat(yy_tn.values())
    y_te = pd.concat(yy_te.values())
    
    return X_tn, X_te, y_tn, y_te

In [18]:
def Domain_Split(X, y, test_domain):
    X_M = X.xs(key = 'M', level = 'Next', drop_level = False)
    X_D = X.xs(key = 'D', level = 'Next', drop_level = False)
    X_H = X.xs(key = 'H', level = 'Next', drop_level = False)
    
    XX = {"M" : X_M, "D" : X_D, "H" : X_H}
    
    y_M = y.xs(key = 'M', level = 'Next', drop_level = False)
    y_D = y.xs(key = 'D', level = 'Next', drop_level = False)
    y_H = y.xs(key = 'H', level = 'Next', drop_level = False)
    
    yy = {"M" : y_M, "D" : y_D, "H" : y_H}
    
    
    X_tn = pd.concat([XX[n] for n in ['M', 'D', 'H'] if n not in test_domain])
    X_te = pd.concat([XX[n] for n in ['M', 'D', 'H'] if n in test_domain])

    y_tn = pd.concat([yy[n] for n in ['M', 'D', 'H'] if n not in test_domain])
    y_te = pd.concat([yy[n] for n in ['M', 'D', 'H'] if n in test_domain])
    
    return X_tn, X_te, y_tn, y_te

In [19]:
def get_next_tensor(index_list):
    next_mapping = {'M': 0, 'D': 1, 'H': 2}
    next_index = [next_mapping[idx[0]] for idx in index_list] 
    next_tensor = torch.tensor(next_index)
    one_hot = torch.nn.functional.one_hot(next_tensor, num_classes=3).float()
    return one_hot

In [20]:
def setRandomSeed(random_seed=0):
    os.environ['PYTHONHASHSEED'] = str(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

In [21]:
class MLP(nn.Module):
    def __init__(self, hidden_dim=64, num_layers=2, next_dim = 3):
        super(MLP, self).__init__()
        layers = []

        input_dim = 4 + next_dim
        output_dim = 1

        layers.append(nn.Linear(input_dim, hidden_dim))
        layers.append(nn.ReLU())

        for _ in range(num_layers - 1):
            layers.append(nn.Linear(hidden_dim, hidden_dim))
            layers.append(nn.ReLU())

        layers.append(nn.Linear(hidden_dim, output_dim))

        self.model = nn.Sequential(*layers)

    def forward(self, x, onehot):
        x_concat = torch.cat([x, onehot], dim=1)
        return self.model(x_concat)

class MAPELoss(nn.Module):
    def __init__(self, epsilon=1e-7):
        super(MAPELoss, self).__init__()
        self.epsilon = epsilon

    def forward(self, y_pred, y_true):
        return torch.mean(torch.abs((y_true - y_pred) / (y_true + self.epsilon))) * 100

In [22]:
class Trainer:
    def __init__(self, model, lr=1e-3, weight_decay=0, epoch=1000, patience=50, random_seed = 0, optimizer = None):
        self.random_seed = random_seed
        #setRandomSeed(self.random_seed)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.criterion = nn.MSELoss()
        self.cri2 = MAPELoss()
        if optimizer is None:
            self.optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
        else:
            self.optimizer = optimizer
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=200, gamma=0.8)
        self.epoch = epoch
        self.patience = patience

        self.train_loss = []
        self.test_loss = []
        self.val_loss = []

    def train(self, train_loader, test_loader=None, val_loader=None):
        best_val_loss = float('inf')
        best_model_state = None
        epochs_no_improve = 0

        for ep in range(1, self.epoch + 1):
            self.model.train()
            for x_batch, onehot, y_batch in train_loader:
                x_batch, onehot, y_batch = x_batch.to(self.device), onehot.to(self.device), y_batch.to(self.device)

                self.optimizer.zero_grad()
                outputs = self.model(x_batch, onehot)
                loss = self.cri2(outputs, y_batch)
                loss.backward()
                self.optimizer.step()

            self.scheduler.step()

            avg_train_loss = self.evaluate(train_loader)
            test_loss = self.evaluate(test_loader) if test_loader else None
            val_loss = self.evaluate(val_loader) if val_loader else None

            self.train_loss.append(avg_train_loss)
            self.test_loss.append(test_loss)
            self.val_loss.append(val_loss)

            if val_loader:
                if val_loss < best_val_loss - 1e-4:
                    best_val_loss = val_loss
                    best_model_state = self.model.state_dict()
                    epochs_no_improve = 0
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= self.patience:
                        print(f"Early stopping at epoch {ep}")
                        break

        if best_model_state:
            self.model.load_state_dict(best_model_state)

    def evaluate(self, data_loader):
        if data_loader is None:
            return None
        self.model.eval()
        total_loss = 0
        with torch.no_grad():
            for x_batch, onehot, y_batch in data_loader:
                x_batch, onehot, y_batch = x_batch.to(self.device), onehot.to(self.device), y_batch.to(self.device)
                outputs = self.model(x_batch, onehot)
                loss = self.cri2(outputs, y_batch)
                total_loss += loss.item()
        return total_loss / len(data_loader)

    def predict(self, x, onehot):
        self.model.eval()
        with torch.no_grad():
            x = x.to(self.device)
            onehot = onehot.to(self.device)
            return self.model(x, onehot)


In [23]:
def plot_results(info, train_loader, val_loader, test_loader, plot = True):
    rs, hid, nl, lr = info
    
    y_true_train = []
    y_pred_train = []
    
    y_true_val = []
    y_pred_val = []
    
    y_true_test = []
    y_pred_test = []

    y_true_test_M = []
    y_pred_test_M = []

    y_true_test_D = []
    y_pred_test_D = []

    y_true_test_H = []
    y_pred_test_H = []
    
    for x_batch, onehot, y_batch in train_loader:
        preds = trainer.predict(x_batch, onehot)
        y_true_train.append(y_batch)
        y_pred_train.append(preds.cpu())
    
    for x_batch, onehot, y_batch in val_loader:
        preds = trainer.predict(x_batch, onehot)
        y_true_val.append(y_batch)
        y_pred_val.append(preds.cpu())
    
    for x_batch, onehot, y_batch in test_loader:
        preds = trainer.predict(x_batch, onehot)
        y_true_test.append(y_batch)
        y_pred_test.append(preds.cpu())
    
        onehot_np = onehot.cpu().numpy()
        y_true_np = y_batch.cpu().numpy()
        y_pred_np = preds.cpu().numpy()
    
        for i in range(len(onehot_np)):
            if np.array_equal(onehot_np[i], [1, 0, 0]):  # 'M'
                y_true_test_M.append(y_true_np[i])
                y_pred_test_M.append(y_pred_np[i])
            elif np.array_equal(onehot_np[i], [0, 1, 0]):  # 'D'
                y_true_test_D.append(y_true_np[i])
                y_pred_test_D.append(y_pred_np[i])
            elif np.array_equal(onehot_np[i], [0, 0, 1]):  # 'H'
                y_true_test_H.append(y_true_np[i])
                y_pred_test_H.append(y_pred_np[i])
    
    y_true_train = torch.cat(y_true_train).numpy()
    y_pred_train = torch.cat(y_pred_train).numpy()
    
    y_true_val = torch.cat(y_true_val).numpy()
    y_pred_val = torch.cat(y_pred_val).numpy()
    
    y_true_test = torch.cat(y_true_test).numpy()
    y_pred_test = torch.cat(y_pred_test).numpy()

    y_true_test_M = np.array(y_true_test_M)
    y_pred_test_M = np.array(y_pred_test_M)
    
    y_true_test_D = np.array(y_true_test_D)
    y_pred_test_D = np.array(y_pred_test_D)
    
    y_true_test_H = np.array(y_true_test_H)
    y_pred_test_H = np.array(y_pred_test_H)
    
    mape_M = mean_absolute_percentage_error(y_true_test_M, y_pred_test_M) * 100 if len(y_true_test_M) > 0 else np.nan
    mape_D = mean_absolute_percentage_error(y_true_test_D, y_pred_test_D) * 100 if len(y_true_test_D) > 0 else np.nan
    mape_H = mean_absolute_percentage_error(y_true_test_H, y_pred_test_H) * 100 if len(y_true_test_H) > 0 else np.nan
    
    train_mape = mean_absolute_percentage_error(y_true_train, y_pred_train) * 100
    val_mape = mean_absolute_percentage_error(y_true_val, y_pred_val) * 100
    test_mape = mean_absolute_percentage_error(y_true_test, y_pred_test) * 100
    print(f"Rs: {rs}, hid: {hid}, {nl} layers, lr: {lr}\n Train MAPE: {train_mape:.2f}%, Val MAPE: {val_mape:.2f}%, Test MAPE: {test_mape:.2f}%")
    print(f"Test MAPE by 'Next': M: {mape_M:.2f}%, D: {mape_D:.2f}%, H: {mape_H:.2f}%")
    if plot == True:
        _ = plt.figure()
        _ = plt.scatter(y_true_train, y_pred_train, label = 'Train')
        _ = plt.scatter(y_true_val, y_pred_val, label = 'Val')
        _ = plt.scatter(y_true_test, y_pred_test, label = 'Test')
        min_val = min(y_true_train.min(), y_true_test.min())
        max_val = max(y_true_train.max(), y_true_test.max())
        _ = plt.plot([min_val, max_val], [min_val, max_val], 'k--', label='Ideal line')
        _ = plt.xlabel('True SOH')
        _ = plt.ylabel('Predicted SOH')
        _ = plt.legend()
        _ = plt.title(f'Random state: {rs}, hid: {hid}, {nl} layer, lr: {lr}')
    return train_mape, val_mape, test_mape, mape_M, mape_D, mape_H

def plot_loss(info, train_loss, val_loss, test_loss):
    rs, hid, nl, lr = info
    _ = plt.figure()
    _ = plt.plot(train_loss, label = 'Train loss')
    _ = plt.plot(val_loss, label = 'Val loss')
    _ = plt.plot(test_loss, label = 'Test loss')
    _ = plt.ylim([0, 2])
    _ = plt.xlabel('Epoch')
    _ = plt.ylabel('Loss')
    _ = plt.legend()
    _ = plt.title(f'Random state: {rs}, hid: {hid}, {nl} layers, lr: {lr}')

In [24]:
random_states = [100, 120, 140, 160, 180]
bs = 12
ep = 1000
hids = [8, 16, 32]
layers = [2, 3, 4, 5, 6]
lrs = [1e-3, 1e-4]

Source_Domain = ['D', 'H']
Target_Domain = ['M']

results_df = pd.DataFrame(columns = ['rs', 'hid', 'nl', 'lr', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE'])

X, y = Get_Data(dat)

X_sc, X_ta, y_sc, y_ta = Domain_Split(X, y, Target_Domain)

for rs, hid, nl, lr in product(random_states, hids, layers, lrs):
    setRandomSeed(rs)

    X_tn, X_te, y_tn, y_te = Even_Split(X_sc, y_sc, Target_Domain, 1/3, rs)
    X_tr, X_va, y_tr, y_va = Even_Split(X_tn, y_tn, Target_Domain, 1/6, rs)

    next_map = {'M': 0, 'D': 1, 'H': 2}
    get_next = lambda idx: torch.nn.functional.one_hot(torch.tensor([next_map[i] for i in idx.get_level_values("Next")]), num_classes=3).float()

    std_scaler = StandardScaler()
    X_tr_std = std_scaler.fit_transform(X_tr)
    X_val_std = std_scaler.transform(X_va)
    X_te_std = std_scaler.transform(X_te)
    
    X_train = torch.Tensor(X_tr_std)
    X_val = torch.Tensor(X_val_std)
    X_test = torch.Tensor(X_te_std)
    
    y_train = torch.Tensor(y_tr["Next_SOH"].values).unsqueeze(1)
    y_val = torch.Tensor(y_va["Next_SOH"].values).unsqueeze(1)
    y_test = torch.Tensor(y_te["Next_SOH"].values).unsqueeze(1)

    next_train = get_next(X_tr.index)
    next_val = get_next(X_va.index)
    next_test = get_next(X_te.index)

    train_loader = DataLoader(TensorDataset(X_train, next_train, y_train), batch_size=bs, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, next_val, y_val), batch_size=bs, shuffle = False)
    test_loader = DataLoader(TensorDataset(X_test, next_test, y_test), batch_size=bs, shuffle = False)
    
    model = MLP(hidden_dim=hid, num_layers=nl)
    trainer = Trainer(model, lr=lr, epoch = ep, random_seed = rs)
    
    trainer.train(train_loader, val_loader, test_loader)

    info = [rs, hid, nl, lr]
    results = plot_results(info, train_loader, val_loader, test_loader, plot = False)

    temp_df = pd.DataFrame(info+list(results)).T
    temp_df.columns = ['rs', 'hid', 'nl', 'lr', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']
    results_df = pd.concat([results_df, temp_df])


Early stopping at epoch 142
Rs: 100, hid: 8, 2 layers, lr: 0.001
 Train MAPE: 0.66%, Val MAPE: 0.97%, Test MAPE: 1.09%
Test MAPE by 'Next': M: nan%, D: 0.80%, H: 1.37%
Rs: 100, hid: 8, 2 layers, lr: 0.0001
 Train MAPE: 0.73%, Val MAPE: 1.50%, Test MAPE: 1.27%
Test MAPE by 'Next': M: nan%, D: 0.77%, H: 1.77%
Early stopping at epoch 228
Rs: 100, hid: 8, 3 layers, lr: 0.001
 Train MAPE: 0.61%, Val MAPE: 0.73%, Test MAPE: 0.84%
Test MAPE by 'Next': M: nan%, D: 0.72%, H: 0.97%
Rs: 100, hid: 8, 3 layers, lr: 0.0001
 Train MAPE: 0.70%, Val MAPE: 1.02%, Test MAPE: 0.94%
Test MAPE by 'Next': M: nan%, D: 0.81%, H: 1.08%
Early stopping at epoch 167
Rs: 100, hid: 8, 4 layers, lr: 0.001
 Train MAPE: 0.78%, Val MAPE: 0.84%, Test MAPE: 0.97%
Test MAPE by 'Next': M: nan%, D: 1.01%, H: 0.93%
Rs: 100, hid: 8, 4 layers, lr: 0.0001
 Train MAPE: 0.61%, Val MAPE: 0.93%, Test MAPE: 1.26%
Test MAPE by 'Next': M: nan%, D: 0.93%, H: 1.59%
Early stopping at epoch 257
Rs: 100, hid: 8, 5 layers, lr: 0.001
 Train M

In [25]:
summary_df = results_df.groupby(['hid', 'nl', 'lr'])[['Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']].mean().reset_index()

summary_df_sorted = summary_df.sort_values(by = 'Val MAPE', ascending = True)
summary_df_sorted

results_df.to_csv(f"MLP_Domain_Source_{Source_Domain}_next_info.csv")
summary_df_sorted.to_csv(f"MLP_Domain_Source_{Source_Domain}_next_info_sum.csv")

,hid,nl,lr,Train MAPE,Val MAPE,Test MAPE,Next M MAPE,Next D MAPE,Next H MAPE
7,8.0,5.0,0.0010,0.638593,0.765966,0.891031,NaN,0.862189,0.919873
3,8.0,3.0,0.0010,0.673940,0.805833,1.065181,NaN,1.017652,1.112709
6,8.0,5.0,0.0001,0.724763,0.807115,0.944921,NaN,0.875983,1.013859
16,16.0,5.0,0.0001,0.542213,0.822736,0.919734,NaN,0.954309,0.885159
18,16.0,6.0,0.0001,0.639304,0.835669,0.856863,NaN,0.814597,0.899128
5,8.0,4.0,0.0010,0.681245,0.854044,0.997318,NaN,0.991803,1.002832
29,32.0,6.0,0.0010,0.529125,0.855680,0.852480,NaN,0.895139,0.809822
4,8.0,4.0,0.0001,0.713490,0.858679,1.055518,NaN,0.943632,1.167404
15,16.0,4.0,0.0010,0.660869,0.863296,0.955679,NaN,0.932096,0.979262
26,32.0,5.0,0.0001,0.548536,0.908169,1.015917,NaN,1.039692,0.992143


In [26]:
best_hyper_parameter = summary_df_sorted.iloc[0][['hid', 'nl', 'lr']]
best_hyper_parameter

hid    8.000
nl     5.000
lr     0.001
Name: 7, dtype: float64

In [28]:
hid = int(best_hyper_parameter['hid'])
nl  = int(best_hyper_parameter['nl'])
lr  = float(best_hyper_parameter['lr'])

results_df_Ta = pd.DataFrame(columns = ['rs', 'hid', 'nl', 'lr', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE'])
results_df_Sc = pd.DataFrame(columns = ['rs', 'hid', 'nl', 'lr', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE'])


for rs in random_states:
    setRandomSeed(rs)

    X_tn, X_te, y_tn, y_te = Even_Split(X_sc, y_sc, Target_Domain, 1/3, rs)
    X_tr, X_va, y_tr, y_va = Even_Split(X_tn, y_tn, Target_Domain, 1/6, rs)

    next_map = {'M': 0, 'D': 1, 'H': 2}
    get_next = lambda idx: torch.nn.functional.one_hot(torch.tensor([next_map[i] for i in idx.get_level_values("Next")]), num_classes=3).float()

    std_scaler = StandardScaler()
    X_tr_std = std_scaler.fit_transform(X_tr)
    X_val_std = std_scaler.transform(X_va)
    X_te_std = std_scaler.transform(X_te)
    
    X_train = torch.Tensor(X_tr_std)
    X_val = torch.Tensor(X_val_std)
    X_test = torch.Tensor(X_te_std)
    
    y_train = torch.Tensor(y_tr["Next_SOH"].values).unsqueeze(1)
    y_val = torch.Tensor(y_va["Next_SOH"].values).unsqueeze(1)
    y_test = torch.Tensor(y_te["Next_SOH"].values).unsqueeze(1)

    next_train = get_next(X_tr.index)
    next_val = get_next(X_va.index)
    next_test = get_next(X_te.index)

    train_loader = DataLoader(TensorDataset(X_train, next_train, y_train), batch_size=bs, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, next_val, y_val), batch_size=bs, shuffle = False)
    test_loader = DataLoader(TensorDataset(X_test, next_test, y_test), batch_size=bs, shuffle = False)
    
    model = MLP(hidden_dim=hid, num_layers=nl)
    trainer = Trainer(model, lr=lr, epoch = ep, random_seed = rs)
    
    trainer.train(train_loader, val_loader, test_loader)

    info = [rs, hid, nl, lr]
    #results = plot_results(info, train_loader, val_loader, test_loader, plot = False)

    trained_model = trainer.model
    
    for p in trained_model.parameters():
        p.requires_grad = False

    first_linear = trained_model.model[0]

    first_linear.weight.requires_grad = True
    first_linear.bias.requires_grad   = True  
    
    learn_idx = {
        'M': -3,
        'D': -2,
        'H': -1
    }

    mask_w = torch.zeros_like(first_linear.weight)
    for ta in Target_Domain:
        mask_w[:, learn_idx[ta]] = 1.0

    mask_b = torch.zeros_like(first_linear.bias)
    for ta in Target_Domain:
        mask_b[learn_idx[ta]] = 1.0

    def grad_hook_weight(grad):
        return grad * mask_w

    def grad_hook_bias(grad):
        return grad * mask_b

    h1 = first_linear.weight.register_hook(grad_hook_weight)
    h2 = first_linear.bias.register_hook(grad_hook_bias)

    optimizer = torch.optim.Adam(
        [p for p in trained_model.parameters() if p.requires_grad],
        lr=lr,
        weight_decay=0.0 
    )
    
    X_ta_tn, X_ta_te, y_ta_tn, y_ta_te = Even_Split(X_ta, y_ta, Source_Domain, 8/9, rs)
    X_ta_tr, X_ta_va, y_ta_tr, y_ta_va = Even_Split(X_ta_tn, y_ta_tn, Source_Domain, 1/6, rs)

    X_ta_tr_std  = std_scaler.transform(X_ta_tr)
    X_ta_val_std = std_scaler.transform(X_ta_va)
    X_ta_te_std  = std_scaler.transform(X_ta_te)

    X_ta_train = torch.Tensor(X_ta_tr_std)
    X_ta_val   = torch.Tensor(X_ta_val_std)
    X_ta_test  = torch.Tensor(X_ta_te_std)
    
    y_ta_train = torch.Tensor(y_ta_tr["Next_SOH"].values).unsqueeze(1)
    y_ta_val   = torch.Tensor(y_ta_va["Next_SOH"].values).unsqueeze(1)
    y_ta_test  = torch.Tensor(y_ta_te["Next_SOH"].values).unsqueeze(1)

    next_ta_train = get_next(X_ta_tr.index)
    next_ta_val   = get_next(X_ta_va.index)
    next_ta_test  = get_next(X_ta_te.index)

    train_ta_loader = DataLoader(TensorDataset(X_ta_train, next_ta_train, y_ta_train), batch_size=bs, shuffle=True)
    val_ta_loader   = DataLoader(TensorDataset(X_ta_val, next_ta_val, y_ta_val), batch_size=bs, shuffle = False)
    test_ta_loader  = DataLoader(TensorDataset(X_ta_test, next_ta_test, y_ta_test), batch_size=bs, shuffle = False)

    trainer = Trainer(trained_model, lr=lr, epoch = ep, random_seed = rs, optimizer = optimizer)
    
    trainer.train(train_ta_loader, val_ta_loader, test_ta_loader)

    info = [rs, hid, nl, lr]

    results_Ta = plot_results(info, train_ta_loader, val_ta_loader, test_ta_loader, plot = False)
    results_Sc = plot_results(info, train_loader, val_loader, test_loader, plot = False)

    temp_df_Ta = pd.DataFrame(info+list(results_Ta)).T
    temp_df_Sc = pd.DataFrame(info+list(results_Sc)).T

    temp_df_Ta.columns = ['rs', 'hid', 'nl', 'lr', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']
    temp_df_Sc.columns = ['rs', 'hid', 'nl', 'lr', 'Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']

    results_df_Ta = pd.concat([results_df_Ta, temp_df_Ta])
    results_df_Sc = pd.concat([results_df_Sc, temp_df_Sc])

    h1.remove()
    h2.remove()


Early stopping at epoch 257
Early stopping at epoch 299
Rs: 100, hid: 8, 5 layers, lr: 0.001
 Train MAPE: 0.63%, Val MAPE: 0.94%, Test MAPE: 1.30%
Test MAPE by 'Next': M: 1.30%, D: nan%, H: nan%
Rs: 100, hid: 8, 5 layers, lr: 0.001
 Train MAPE: 1.51%, Val MAPE: 1.44%, Test MAPE: 1.84%
Test MAPE by 'Next': M: nan%, D: 1.46%, H: 2.23%
Early stopping at epoch 180
Early stopping at epoch 448
Rs: 120, hid: 8, 5 layers, lr: 0.001
 Train MAPE: 0.80%, Val MAPE: 0.74%, Test MAPE: 1.13%
Test MAPE by 'Next': M: 1.13%, D: nan%, H: nan%
Rs: 120, hid: 8, 5 layers, lr: 0.001
 Train MAPE: 0.90%, Val MAPE: 0.80%, Test MAPE: 1.00%
Test MAPE by 'Next': M: nan%, D: 1.02%, H: 0.98%
Early stopping at epoch 209
Early stopping at epoch 611
Rs: 140, hid: 8, 5 layers, lr: 0.001
 Train MAPE: 0.49%, Val MAPE: 1.00%, Test MAPE: 1.11%
Test MAPE by 'Next': M: 1.11%, D: nan%, H: nan%
Rs: 140, hid: 8, 5 layers, lr: 0.001
 Train MAPE: 0.88%, Val MAPE: 1.12%, Test MAPE: 0.97%
Test MAPE by 'Next': M: nan%, D: 1.00%, H: 0

In [29]:
summary_df_Ta = results_df_Ta.groupby(['hid', 'nl', 'lr'])[['Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']].mean().reset_index()
summary_df_Sc = results_df_Sc.groupby(['hid', 'nl', 'lr'])[['Train MAPE', 'Val MAPE', 'Test MAPE', 'Next M MAPE', 'Next D MAPE', 'Next H MAPE']].mean().reset_index()

summary_df_Sc
summary_df_Ta

,hid,nl,lr,Train MAPE,Val MAPE,Test MAPE,Next M MAPE,Next D MAPE,Next H MAPE
0,8.0,5.0,0.001,1.022326,1.053898,1.231049,NaN,1.146601,1.315498


,hid,nl,lr,Train MAPE,Val MAPE,Test MAPE,Next M MAPE,Next D MAPE,Next H MAPE
0,8.0,5.0,0.001,0.718268,0.984611,1.201515,1.201515,NaN,NaN
